In [1]:
!pip install faiss-cpu open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44

In [ ]:

import os
os.environ["HF_TOKEN"] = ""
from huggingface_hub import login
login(os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# =========================================================
# Keyframe Embedding by Index JSON (OpenCLIP ViT-bigG-14)
# - Load keyframe paths from JSON index file
# - Split by NUM_PARTS / PART_ID (1-based)
# - EXTRA: Split PART 12 into 6 sub-parts (SUB_PART_ID)
# - Embedding (fp16 on CUDA), normalize (L2)
# - Build FAISS IndexFlatIP (cosine via normalized vectors)
# - Save: keyframes_bigG_part_XXX_sub_YY.bin + paths_XXX_sub_YY.txt
# =========================================================

import os
import json
from typing import List
from PIL import Image
from tqdm import tqdm
import numpy as np
import faiss
import torch
import open_clip

# -------------------------
# CONFIG
# -------------------------
INDEX_JSON_PATH = "/kaggle/input/irkeyframeindex/keyframe_index_kaggle.json"
OUTPUT_DIR = "/kaggle/working"

MODEL_NAME = "ViT-bigG-14"
PRETRAINED = "laion2b_s39b_b160k"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

NUM_PARTS = 12       # tổng số phần chính
PART_ID   = 12       # phần bạn muốn chạy (1..12)

# --- SUB SPLIT FOR PART 12 ---
ENABLE_SUB_SPLIT = True   # bật chia nhỏ part 12
SUB_PARTS = 6             # chia nhỏ thành 6 phần
SUB_PART_ID = 6          # chạy phần nhỏ thứ mấy (1..6)

BATCH_SIZE = 64

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------
# Helpers
# -------------------------
def load_keyframe_paths_from_json(json_path: str) -> List[str]:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    sorted_keys = sorted(data.keys(), key=lambda x: int(x))
    return [data[k] for k in sorted_keys]

def split_by_parts(items: List[str], num_parts: int, part_id: int) -> List[str]:
    total = len(items)
    size = total // num_parts
    rem  = total % num_parts
    pid = max(1, min(part_id, num_parts))
    start = (size + 1) * (pid - 1) if pid <= rem else rem * (size + 1) + (pid - rem - 1) * size
    end = start + (size + 1 if pid <= rem else size)
    return items[start:end]

def split_by_parts_simple(items, num_parts, part_id):
    size = len(items) // num_parts
    rem = len(items) % num_parts
    pid = max(1, min(part_id, num_parts))
    start = (size + 1) * (pid - 1) if pid <= rem else rem * (size + 1) + (pid - rem - 1) * size
    end = start + (size + 1 if pid <= rem else size)
    return items[start:end]

def normalize_rows(x: np.ndarray) -> np.ndarray:
    x = x.astype("float32", copy=False)
    norms = np.linalg.norm(x, axis=1, keepdims=True) + 1e-12
    x /= norms
    return x

# -------------------------
# Load model
# -------------------------
print(f"🚀 Loading OpenCLIP {MODEL_NAME} ({PRETRAINED}) on {DEVICE} ...")
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name=MODEL_NAME,
    pretrained=PRETRAINED,
    device=DEVICE,
)
model = model.to(DEVICE, dtype=DTYPE).eval()
print("✅ Model ready.\n")

# -------------------------
# Load keyframe paths from JSON
# -------------------------
print(f"📂 Loading keyframe paths from {INDEX_JSON_PATH} ...")
all_keyframe_paths = load_keyframe_paths_from_json(INDEX_JSON_PATH)
print(f"📊 Total keyframes in index: {len(all_keyframe_paths):,}")

# -------------------------
# Main PART split
# -------------------------
selected_paths = split_by_parts(all_keyframe_paths, NUM_PARTS, PART_ID)
print(f"🎯 PART {PART_ID}/{NUM_PARTS} → {len(selected_paths):,} keyframes before sub-split")

# -------------------------
# OPTIONAL SUB-SPLIT FOR PART 12
# -------------------------
if PART_ID == 12 and ENABLE_SUB_SPLIT:
    selected_paths = split_by_parts_simple(selected_paths, SUB_PARTS, SUB_PART_ID)
    print(f"🔹 SUB-PART {SUB_PART_ID}/{SUB_PARTS} → {len(selected_paths):,} keyframes after sub-split\n")
else:
    print("🔸 No sub-split applied\n")

if not selected_paths:
    raise SystemExit("❌ No keyframes found in selected part.")

# -------------------------
# Embedding
# -------------------------
EMBS = []
accepted_paths = []

for i in tqdm(range(0, len(selected_paths), BATCH_SIZE), desc="🔄 Embedding", dynamic_ncols=True):
    batch_paths = selected_paths[i:i+BATCH_SIZE]
    batch_imgs = []
    batch_valid_paths = []
    
    for p in batch_paths:
        try:
            img = Image.open(p).convert("RGB")
            batch_imgs.append(preprocess(img))
            batch_valid_paths.append(p)
        except:
            continue

    if not batch_imgs:
        continue

    batch = torch.stack(batch_imgs).to(DEVICE, dtype=DTYPE)
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=DTYPE):
        feats = model.encode_image(batch)
        feats = feats / feats.norm(dim=-1, keepdim=True)
    EMBS.append(feats.cpu().numpy())
    accepted_paths.extend(batch_valid_paths)

    if DEVICE == "cuda":
        torch.cuda.empty_cache()

if not EMBS:
    raise SystemExit("❌ No embeddings produced.")

EMBS = np.concatenate(EMBS, axis=0).astype("float32")
EMBS = normalize_rows(EMBS)
print("✅ Embedding shape:", EMBS.shape)

assert len(accepted_paths) == EMBS.shape[0]

# -------------------------
# Build FAISS index (cosine via IP)
# -------------------------
index = faiss.IndexFlatIP(EMBS.shape[1])
index.add(EMBS)

bin_path   = os.path.join(
    OUTPUT_DIR,
    f"keyframes_bigG_part_{PART_ID:03d}_sub_{SUB_PART_ID:02d}.bin"
)
paths_path = os.path.join(
    OUTPUT_DIR,
    f"keyframes_bigG_paths_part_{PART_ID:03d}_sub_{SUB_PART_ID:02d}.txt"
)

faiss.write_index(index, bin_path)
with open(paths_path, "w", encoding='utf-8') as f:
    f.write("\n".join(accepted_paths))

print("\n🎉 DONE")
print("📦 Index  →", bin_path)
print("📄 Paths  →", paths_path)
print("🔢 Vectors:", EMBS.shape)
print(f"📈 Processed: {len(accepted_paths):,}/{len(selected_paths):,} images")


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

🚀 Loading OpenCLIP ViT-bigG-14 (laion2b_s39b_b160k) on cuda ...


open_clip_model.safetensors:   0%|          | 0.00/10.2G [00:00<?, ?B/s]

✅ Model ready.

📂 Loading keyframe paths from /kaggle/input/irkeyframeindex/keyframe_index_kaggle.json ...
📊 Total keyframes in index: 702,648
🎯 PART 12/12 → 58,554 keyframes before sub-split
🔹 SUB-PART 6/6 → 9,759 keyframes after sub-split



🔄 Embedding:   0%|          | 0/153 [00:00<?, ?it/s]/tmp/ipykernel_20/352487735.py:136: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(DEVICE=="cuda"), dtype=DTYPE):
🔄 Embedding: 100%|██████████| 153/153 [14:33<00:00,  5.71s/it]

✅ Embedding shape: (9759, 1280)

🎉 DONE
📦 Index  → /kaggle/working/keyframes_bigG_part_012_sub_06.bin
📄 Paths  → /kaggle/working/keyframes_bigG_paths_part_012_sub_06.txt
🔢 Vectors: (9759, 1280)
📈 Processed: 9,759/9,759 images
